In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('information.csv')

In [3]:
data.rename(columns={'country': 'Country', 'city': 'City', 'temp': 'Temp(°C)', 'wind': 'Wind(km/h)', 'visibility': 'Visibility(km)', 'humidity': 'Humidity(%)', 'dewpoint': 'Dewpoint(°C)', 
                    'rainchance': 'Rainchance(%)', 'rainamount': 'Rainamount(mm)', 'ntemp': 'Temp next day(°C)', 'nwind': 'Wind next day(km/h)', 'nhumidity': 'Humidity next day(%)', 
                    'nrainchance': 'Rainchance next day(%)', 'nrainamount': 'Rainamount next day(mm)'}, inplace=True)

In [4]:
data

,Country,City,Temp(°C),weather,Wind(km/h),Visibility(km),pressure,Humidity(%),Dewpoint(°C),Rainchance(%),Rainamount(mm),Temp next day(°C),Wind next day(km/h),Humidity next day(%),Rainchance next day(%),Rainamount next day(mm)
0,Albania,Tirana,12.0,Light rain. Partly sunny.,13,NaN,1002.0,72.0,7.0,48,0.0,9,6,53,0,0.0
1,Algeria,Algiers,18.0,Sunny.,13,10.0,1023.0,59.0,10.0,0,NaN,18,11,63,0,NaN
2,Angola,Luanda,30.0,Scattered clouds.,7,NaN,1010.0,66.0,23.0,76,6.0,31,13,68,15,0.0
3,Antigua and Barbuda,Saint Johns,26.0,Scattered showers. Scattered clouds.,20,NaN,1017.0,74.0,21.0,17,NaN,28,19,72,22,NaN
4,Argentina,Buenos Aires,28.0,Sunny.,9,16.0,1009.0,55.0,18.0,8,NaN,30,14,53,0,0.0
5,Armenia,Yerevan,7.0,Passing clouds.,No wind,10.0,1022.0,40.0,-6.0,13,NaN,6,3,57,31,NaN
6,Australia,Canberra,23.0,Clear.,9,NaN,1010.0,36.0,7.0,0,NaN,31,23,24,0,0.0
7,Austria,Vienna,2.0,Light rain. Overcast.,7,8.0,1009.0,86.0,0.0,32,4.0,1,11,80,32,2.0
8,Azerbaijan,Baku,7.0,Fog.,44,5.0,1013.0,87.0,5.0,0,NaN,6,14,82,5,NaN
9,Bahamas,Nassau,22.0,Broken clouds.,7,NaN,1022.0,78.0,18.0,25,NaN,26,16,72,7,0.0
